In [1]:
import torch
import math

In [2]:

# Bài 1 
# Ma trận đầu vào A (hàng = Thời tiết, cột = Địa điểm)
A = torch.tensor([
    [1, 2, 3, 4],   # Nắng
    [3, 4, 5, 6],   # Mưa
    [2, 3, 5, 6],   # Âm u
    [1, 3, 5, 7]    # Lạnh
], dtype=torch.float32)

# ---------------------------------------------------------
# 1) Tính phân phối xác suất chung P(Weather, Location)
# ---------------------------------------------------------

# Tổng tất cả phần tử trong ma trận (để chuẩn hóa về xác suất)
total_sum = A.sum()

# Chia từng phần tử cho tổng → ma trận xác suất chung
P_joint = A / total_sum


# ---------------------------------------------------------
# 2) Tính phân phối biên theo THỜI TIẾT: P(Weather)
# ---------------------------------------------------------
# Cộng theo từng hàng (dim=1 = tổng theo cột)
P_weather = P_joint.sum(dim=1)


# ---------------------------------------------------------
# 3) Tính phân phối biên theo ĐỊA ĐIỂM: P(Location)
# ---------------------------------------------------------
# Cộng theo từng cột (dim=0 = tổng theo hàng)
P_location = P_joint.sum(dim=0)


# ---------------------------------------------------------
# 4) Gán nhãn cho rõ ràng (không bắt buộc, nhưng giúp dễ đọc)
weather_labels = ['Nắng', 'Mưa', 'Âm u', 'Lạnh']
location_labels = ['Rất gần', 'Gần', 'Xa', 'Rất xa']


# ---------------------------------------------------------
# 5) In kết quả
# ---------------------------------------------------------

print("=== Phân phối xác suất chung P(Weather, Location) ===")
print(P_joint)

print("\n=== Phân phối biên P(Weather) ===")
for lbl, prob in zip(weather_labels, P_weather):
    print(f"{lbl}: {prob:.4f}")

print("\n=== Phân phối biên P(Location) ===")
for lbl, prob in zip(location_labels, P_location):
    print(f"{lbl}: {prob:.4f}")


#######################3######################################

=== Phân phối xác suất chung P(Weather, Location) ===
tensor([[0.0167, 0.0333, 0.0500, 0.0667],
        [0.0500, 0.0667, 0.0833, 0.1000],
        [0.0333, 0.0500, 0.0833, 0.1000],
        [0.0167, 0.0500, 0.0833, 0.1167]])

=== Phân phối biên P(Weather) ===
Nắng: 0.1667
Mưa: 0.3000
Âm u: 0.2667
Lạnh: 0.2667

=== Phân phối biên P(Location) ===
Rất gần: 0.1167
Gần: 0.2000
Xa: 0.3000
Rất xa: 0.3833


In [ ]:
# ---------------------------------------------------------
# 2) HÀM TÍNH XÁC SUẤT MULTINOMIAL
#    P(X1 = x1, ..., Xm = xm)
#    n  : tổng số phép thử
#    counts : [x1, x2, ..., xm]
#    probs  : [p1, p2, ..., pm]
# ---------------------------------------------------------
def multinomial_prob(n, counts, probs):
    # Kiểm tra tổng số lần quan sát có đúng n không
    if sum(counts) != n:
        raise ValueError("Tổng các x_i trong 'counts' phải bằng n")
    
    # Kiểm tra vector xác suất có tổng ~ 1 không
    if abs(sum(probs) - 1.0) > 1e-9:
        raise ValueError("Tổng các p_i trong 'probs' phải bằng 1")

    # --- Hệ số tổ hợp: n! / (x1! x2! ... xm!)
    coef = math.factorial(n)
    for x in counts:
        coef /= math.factorial(x)

    # --- Phần xác suất: p1^x1 * p2^x2 * ... * pm^xm
    prob_term = 1.0
    for x, p in zip(counts, probs):
        prob_term *= (p ** x)

    return coef * prob_term


# Ví dụ: n = 5 phép thử, 3 loại:
#   loại 1 xảy ra 2 lần
#   loại 2 xảy ra 2 lần
#   loại 3 xảy ra 1 lần
# xác suất mỗi loại: p = [0.2, 0.3, 0.5]
n = 5
counts = [2, 2, 1]
probs = [0.2, 0.3, 0.5]

p_mult = multinomial_prob(n, counts, probs)
print("P_multinomial =", p_mult)

0.05400000000000001


In [ ]:
def binomial_prob(n, k, p):
    """
    X ~ Binomial(n, p)
    Trả về P(X = k)
    """
    q = 1.0 - p  # xác suất thất bại
    # C(n, k) = n! / (k!(n-k)!)
    comb = math.factorial(n) // (math.factorial(k) * math.factorial(n - k))
    return comb * (p ** k) * (q ** (n - k))


# Ví dụ:  n = 10, p = 0.3, xác suất có đúng 4 lần thành công:
print("P_binomial =", binomial_prob(10, 4, 0.3))


P_binomial = 0.2001209489999999


In [5]:
# =========================
# CÂU 3
# =========================
# Xác suất tán đổ 1 cô gái: p = 0.1
# Tìm n nhỏ nhất sao cho: P(có ít nhất 1 cô đổ) > 0.5
# P(ít nhất 1) = 1 - (1-p)^n = 1 - 0.9^n > 0.5

p = 0.1
threshold = 0.5

n = 1
while True:
    prob_have_gf = 1 - (1 - p) ** n
    if prob_have_gf > threshold:
        break
    n += 1

print("Câu 3:")
print(f"  Cần tiếp cận ít nhất n = {n} cô gái")
print(f"  Xác suất có bạn gái = {prob_have_gf:.4f}")
print("-" * 50)

Câu 3:
  Cần tiếp cận ít nhất n = 7 cô gái
  Xác suất có bạn gái = 0.5217
--------------------------------------------------


In [7]:

# =========================
# CÂU 4
# =========================
# Thời gian chờ xe bus ~ Exponential(mean = 7 phút)
# Tham số lambda = 1 / mean
# P(T < 5) = 1 - e^{-lambda * 5}

mean_wait = 7.0
lam = 1.0 / mean_wait

t = 5.0
prob_less_5 = 1 - math.exp(-lam * t)

print(f"  Xác suất học sinh chờ < 5 phút ≈ {prob_less_5:.4f} ({prob_less_5*100:.2f}%)")
print("-" * 50)


  Xác suất học sinh chờ < 5 phút ≈ 0.5105 (51.05%)
--------------------------------------------------


In [ ]:


# =========================
# CÂU 5
# =========================
# Hai thùng bi, mỗi thùng 10 viên
# Trái: 3 xanh, 7 khác  → P(xanh | trái) = 3/10
# Phải: 5 xanh, 5 khác → P(xanh | phải) = 5/10
# Chọn thùng ngẫu nhiên → P(trái) = P(phải) = 1/2
# Hỏi P(trái | đã rút được bi xanh)?  (Bayes)

p_blue_given_left = 3 / 10
p_blue_given_right = 5 / 10
p_left = 0.5
p_right = 0.5

# Xác suất rút được bi xanh (toàn phần)
p_blue = p_blue_given_left * p_left + p_blue_given_right * p_right

# Bayes: P(left | blue)
p_left_given_blue = p_blue_given_left * p_left / p_blue

print("Câu 5:")
print(f"  Xác suất bi xanh đến từ thùng bên trái ≈ {p_left_given_blue:.4f} ({p_left_given_blue*100:.2f}%)")
print("-" * 50)




Câu 5:
  Xác suất bi xanh đến từ thùng bên trái ≈ 0.3750 (37.50%)
--------------------------------------------------


In [9]:

# =========================
# CÂU 6
# =========================
# Hai cửa hàng vé số.
# A: xác suất trúng [giải nhất, nhì, ba, KK] = [0.005, 0.01, 0.015, 0.02]
# B:                                  = [0.006, 0.009, 0.017, 0.01]
# Giá trị giải: [100, 50, 30, 20] (triệu)
# Kỳ vọng E = Σ p_i * v_i

probs_A = [0.005, 0.01, 0.015, 0.02]
probs_B = [0.006, 0.009, 0.017, 0.01]
values = [100, 50, 30, 20]   # triệu

E_A = sum(p * v for p, v in zip(probs_A, values))
E_B = sum(p * v for p, v in zip(probs_B, values))

choose = "A" if E_A > E_B else "B"

print("Câu 6:")
print(f"  Kỳ vọng cửa hàng A = {E_A:.3f} triệu")
print(f"  Kỳ vọng cửa hàng B = {E_B:.3f} triệu")
print(f"  Nên mua vé số ở cửa hàng: {choose}")
print("-" * 50)



Câu 6:
  Kỳ vọng cửa hàng A = 1.850 triệu
  Kỳ vọng cửa hàng B = 1.760 triệu
  Nên mua vé số ở cửa hàng: A
--------------------------------------------------


In [ ]:


# =========================
# CÂU 7
# =========================
# Người đi bộ 5 phút ra bến xe.
# Thời gian xe bus đến T ~ N(μ, σ^2)
# μ = 7:30 AM, phương sai = 10 phút^2 ⇒ σ = sqrt(10)
# Muốn P(bắt kịp xe) = P(T >= t_arrive) = 0.95
# → P(T < t_arrive) = 0.05
# Với Z chuẩn: P(Z < -1.645) = 0.05
# t_arrive = μ + z * σ  với z = -1.645
# Sau đó t_depart = t_arrive - 5 phút

# Đổi giờ 7:30 thành phút tính từ 0:00 để dễ cộng trừ
mu_hour = 7
mu_minute = 30
mu_total_min = mu_hour * 60 + mu_minute  # 7*60 + 30 = 450

variance = 10.0
sigma = math.sqrt(variance)

z_005 = -1.645  # quantile ~ N(0,1) cho mức 0.05

t_arrive_min = mu_total_min + z_005 * sigma
t_depart_min = t_arrive_min - 5.0

# Chuyển ngược lại sang giờ:phút
dep_hour = int(t_depart_min // 60)
dep_min = t_depart_min - dep_hour * 60

print("Câu 7:")
print(f"  Nên có mặt ở bến lúc ≈ {int(t_arrive_min//60)}h{int(t_arrive_min%60):02d}")
print(f"  Thời gian nên rời nhà ≈ {dep_hour}h{int(dep_min):02d} (xác suất bắt kịp ≈ 95%)")
print("-" * 50)